# Test Notebook
## Play with functions and new APIs before adding them to the test lineup.

In [1]:
import matplotlib.pyplot as plt

def print_centroids(c):
    for index, image in enumerate(c):
        image = image.numpy().astype(float).reshape(64,64,3) / 255
        print(f"Centroid {index}:")
        plt.imshow(image)
        plt.show()
        
def print_centroid(c):
    image = c
    image = image.numpy().astype(float).reshape(64,64,3) / 255
    plt.imshow(image)
    plt.show()

In [2]:
import util
import tensorflow as tf
ds = util.build_frames_dataset().map(lambda x: tf.reshape(x, [-1])).shuffle(20000, seed=0)

Metal device set to: Apple M1


2022-09-15 20:48:45.162234: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-15 20:48:45.162330: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
k = 10
# Initialize k with the first few random points
centroids = tf.Variable(tf.cast(tf.stack([f for f in ds.take(k)]), tf.float32))
next_round_centroids = tf.Variable(centroids)
last_entropy = None
for _ in range(10):
    # Keep track of counts of points assigned to each centroid
    centroid_counts = tf.ones(k, dtype=tf.int32)
    running_norms = 0
    for batch in ds.batch(10000):
        # Assign samples to the centroid that each is closest to
        diffs = centroids - tf.expand_dims(tf.cast(batch, tf.float32), axis=1)
        norms = tf.reshape(diffs, [-1, diffs.shape[2]])
        norms = tf.reshape(tf.norm(norms, axis=1), diffs.shape[:2])
        assigned_centroids = tf.cast(tf.math.argmin(norms, axis=1), tf.int32)
        running_norms += tf.reduce_sum(tf.reduce_min(norms, axis=1))
        # Calculate updated counts for each centroid (for calculating online average)
        batch_centroid_counts = tf.math.bincount(assigned_centroids, minlength=k)
        new_centroid_counts = centroid_counts + batch_centroid_counts
        # Update the next round centroids as mean of all assigned points
        c_update_sum = []
        for c_index in range(k):
            c_update_sum.append(tf.reduce_sum(tf.cast(tf.where(tf.equal(assigned_centroids[:,tf.newaxis], c_index), batch, [0]), tf.int32), axis=0))
        c_update_sum = tf.cast(tf.stack(c_update_sum), tf.float32)
        c_update_mean = tf.math.divide_no_nan(c_update_sum, tf.cast(batch_centroid_counts[:,tf.newaxis], tf.float32))
        update_fractions = tf.cast(centroid_counts / new_centroid_counts, tf.float32)
        next_round_centroids.assign(next_round_centroids * update_fractions[:,tf.newaxis] + c_update_mean * (1-update_fractions)[:,tf.newaxis])
        centroid_counts = new_centroid_counts
    # Determine convergence criteria.  Break if converged.
    entropy = running_norms
    if last_entropy != None and last_entropy == entropy:
        break
    last_entropy = entropy
    print(entropy.numpy())
    # Update centroids from next_round_centroids
    centroids.assign(next_round_centroids)

KeyboardInterrupt: 